In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from utils import *

In [5]:
ce_loss = nn.CrossEntropyLoss(reduction='sum')

In [4]:
model_t = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(32, 64, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(3136, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [8]:
batch_size = 64
trainloader, testloader, classes = load_CIFAR()

Files already downloaded and verified
Files already downloaded and verified


In [9]:
def criterion_p(op, y, model_p):
    params = list(model_p.parameters())
    return ce_loss(op, y) + (1/len(params))*(torch.norm(torch.cat([param.view(-1) for param in params]))) #+ torch.sum((1/len(params))*torch.exp(torch.abs(torch.cat([param.view(-1) for param in params]))))

In [10]:
def criterion_base(op, y, model_p):
    return nn.CrossEntropyLoss(reduction='sum')(op, y)

In [8]:
loss_history_base = train_model(model_t, trainloader, criterion_base, optimizer = optim.Adam(list(model_t.parameters()), lr=3e-4), epochs=30)

[1,   300] loss: 104.610
[1,   600] loss: 88.707
[2,   300] loss: 70.791
[2,   600] loss: 55.179
[3,   300] loss: 58.114
[3,   600] loss: 52.912
[4,   300] loss: 58.328
[4,   600] loss: 52.296
[5,   300] loss: 48.578
[5,   600] loss: 44.957
[6,   300] loss: 35.368
[6,   600] loss: 48.719
[7,   300] loss: 32.967
[7,   600] loss: 31.847
[8,   300] loss: 26.959
[8,   600] loss: 46.559
[9,   300] loss: 32.831
[9,   600] loss: 25.691
[10,   300] loss: 25.837
[10,   600] loss: 28.721
[11,   300] loss: 15.989
[11,   600] loss: 19.572
[12,   300] loss: 14.056
[12,   600] loss: 13.411
[13,   300] loss: 20.452
[13,   600] loss: 12.801
[14,   300] loss: 5.306
[14,   600] loss: 8.824
[15,   300] loss: 4.021
[15,   600] loss: 7.974
[16,   300] loss: 4.622
[16,   600] loss: 6.827
[17,   300] loss: 4.869
[17,   600] loss: 10.545
[18,   300] loss: 3.993
[18,   600] loss: 10.413
[19,   300] loss: 4.356
[19,   600] loss: 13.024
[20,   300] loss: 4.642
[20,   600] loss: 19.049
[21,   300] loss: 3.343
[21

In [9]:
model_t_v2 = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [10]:
# train v2 with only 2 conv layers
loss_history_test = train_model(model_t_v2, trainloader, criterion_base, optimizer = optim.Adam(list(model_t_v2.parameters()), lr=3e-4), epochs=30)

[1,   300] loss: 103.257
[1,   600] loss: 99.812
[2,   300] loss: 80.348
[2,   600] loss: 92.498
[3,   300] loss: 77.158
[3,   600] loss: 48.082
[4,   300] loss: 70.882
[4,   600] loss: 62.246
[5,   300] loss: 65.975
[5,   600] loss: 40.804
[6,   300] loss: 37.586
[6,   600] loss: 44.484
[7,   300] loss: 23.264
[7,   600] loss: 45.321
[8,   300] loss: 35.745
[8,   600] loss: 34.502
[9,   300] loss: 29.021
[9,   600] loss: 32.116
[10,   300] loss: 15.186
[10,   600] loss: 20.507
[11,   300] loss: 12.921
[11,   600] loss: 11.523
[12,   300] loss: 13.408
[12,   600] loss: 9.024
[13,   300] loss: 10.838
[13,   600] loss: 16.665
[14,   300] loss: 8.623
[14,   600] loss: 9.927
[15,   300] loss: 7.483
[15,   600] loss: 2.652
[16,   300] loss: 6.615
[16,   600] loss: 5.945
[17,   300] loss: 3.486
[17,   600] loss: 9.223
[18,   300] loss: 6.653
[18,   600] loss: 2.350
[19,   300] loss: 0.792
[19,   600] loss: 10.142
[20,   300] loss: 3.318
[20,   600] loss: 3.033
[21,   300] loss: 2.922
[21,   

In [18]:
# ! conda list
from sklearn.metrics import *
y_v1, pred_y_v1 = get_predictions_ys(model_t, testloader)
print("3-conv-layer net f1 score: " + str(f1_score(y_v1, pred_y_v1, average=None)))
print("3-conv-layer net accuracy: " + str(accuracy_score(y_v1, pred_y_v1)))

y_v2, pred_y_v2 = get_predictions_ys(model_t_v2, testloader)
print("2-conv-layer net f1 score: " + str(f1_score(y_v2, pred_y_v2, average=None)))
print("2-conv-layer net accuracy: " + str(accuracy_score(y_v2, pred_y_v2)))

3-conv-layer net f1 score: [0.74152953 0.81813701 0.62881679 0.49481481 0.64480874 0.59494949
 0.77165354 0.75931842 0.78460128 0.75465498]
3-conv-layer net accuracy: 0.6987
2-conv-layer net f1 score: [0.73837209 0.78554217 0.5707989  0.48594569 0.61942257 0.58224016
 0.71584461 0.74176955 0.80347116 0.73847803]
2-conv-layer net accuracy: 0.6789


In [19]:
print("Stats for 3-conv layer")
print_stats(model_t)

print("Stats for 2-conv layer")
print_stats(model_t_v2)

Stats for 3-conv layer
Mean
0.040802382
standard deviation
0.032516804
Sparsity
0.001633409380589741
Stats for 2-conv layer
Mean
0.040571373
standard deviation
0.03255999
Sparsity
0.0016894062939450901


In [22]:
model_autoencoded = nn.Sequential(
    nn.Conv2d(3, 16, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Conv2d(16, 32, 3),
    nn.FractionalMaxPool2d(2, output_ratio=(1/np.sqrt(2))),
    nn.Flatten(),
    nn.Linear(5408, 1152),
    nn.ReLU(),
    nn.Linear(1152, 512),
    nn.ReLU(),
    nn.Linear(512, 128),
    nn.ReLU(),
    nn.Linear(128, 10)
)

In [23]:
# train v2 with only 2 conv layers
encoder_loss_history_test = train_model_self_reg(model_autoencoded, trainloader, criterion_base, epochs=30)

[1,   300] loss: 106.260
[1,   600] loss: 92.253
[2,   300] loss: 83.559
[2,   600] loss: 67.769
[3,   300] loss: 64.695
[3,   600] loss: 63.473
[4,   300] loss: 69.090
[4,   600] loss: 50.230
[5,   300] loss: 46.203
[5,   600] loss: 45.633
[6,   300] loss: 42.268
[6,   600] loss: 39.803
[7,   300] loss: 33.558
[7,   600] loss: 28.456
[8,   300] loss: 24.236
[8,   600] loss: 15.206
[9,   300] loss: 32.541
[9,   600] loss: 19.956
[10,   300] loss: 9.630
[10,   600] loss: 18.619
[11,   300] loss: 11.118
[11,   600] loss: 23.304
[12,   300] loss: 2.913
[12,   600] loss: 18.697
[13,   300] loss: 5.816
[13,   600] loss: 15.566
[14,   300] loss: 4.608
[14,   600] loss: 1.940
[15,   300] loss: 6.537
[15,   600] loss: 3.817
[16,   300] loss: 2.475
[16,   600] loss: 6.695
[17,   300] loss: 12.743
[17,   600] loss: 5.929
[18,   300] loss: 5.560
[18,   600] loss: 9.534
[19,   300] loss: 9.308
[19,   600] loss: 6.872
[20,   300] loss: 1.696
[20,   600] loss: 14.989
[21,   300] loss: 2.355
[21,   6

In [26]:
print("Stats for encoded layer")
print_stats(model_autoencoded)

y_enc, pred_y_enc = get_predictions_ys(model_autoencoded, testloader)
print("encoded-layer net f1 score: " + str(f1_score(y_enc, pred_y_enc, average=None)))
print("encoded-layer net accuracy: " + str(accuracy_score(y_enc, pred_y_enc)))

y_v1, pred_y_v1 = get_predictions_ys(model_t, testloader)
print("3-conv-layer net f1 score: " + str(f1_score(y_v1, pred_y_v1, average=None)))
print("3-conv-layer net accuracy: " + str(accuracy_score(y_v1, pred_y_v1)))

y_v2, pred_y_v2 = get_predictions_ys(model_t_v2, testloader)
print("2-conv-layer net f1 score: " + str(f1_score(y_v2, pred_y_v2, average=None)))
print("2-conv-layer net accuracy: " + str(accuracy_score(y_v2, pred_y_v2)))

Stats for encoded layer
Mean
0.040562984
standard deviation
0.032107852
Sparsity
0.0015999022854130515
encoded-layer net f1 score: [0.71361502 0.78433367 0.58363731 0.49614644 0.60639361 0.55636896
 0.73207931 0.71525247 0.78912214 0.72831816]
encoded-layer net accuracy: 0.6701


NameError: name 'model_t' is not defined

In [27]:
encoder_loss_history__mse_test = train_model_self_reg(model_autoencoded, trainloader, criterion_base, epochs=30)

[1,   300] loss: 13.080
[1,   600] loss: 4.194
[2,   300] loss: 1.871
[2,   600] loss: 5.305
[3,   300] loss: 1.003
[3,   600] loss: 3.374
[4,   300] loss: 1.097
[4,   600] loss: 1.832
[5,   300] loss: 1.238
[5,   600] loss: 1.315
[6,   300] loss: 4.913
[6,   600] loss: 0.178
[7,   300] loss: 2.337
[7,   600] loss: 2.275
[8,   300] loss: 0.196
[8,   600] loss: 12.094
[9,   300] loss: 0.837
[9,   600] loss: 0.340
[10,   300] loss: 5.283
[10,   600] loss: 0.730
[11,   300] loss: 5.674
[11,   600] loss: 3.058
[12,   300] loss: 1.289
[12,   600] loss: 0.688
[13,   300] loss: 3.814
[13,   600] loss: 0.569
[14,   300] loss: 2.216
[14,   600] loss: 0.276
[15,   300] loss: 4.023
[15,   600] loss: 6.348
[16,   300] loss: 0.122
[16,   600] loss: 2.820
[17,   300] loss: 3.682
[17,   600] loss: 8.774
[18,   300] loss: 0.285
[18,   600] loss: 8.966
[19,   300] loss: 1.230
[19,   600] loss: 4.512
[20,   300] loss: 6.469
[20,   600] loss: 6.002
[21,   300] loss: 3.267
[21,   600] loss: 0.045
[22,   3